
# ReAct Agent with LangGraph - Reasoning and Acting Pattern

This code creates a **ReAct (Reasoning and Acting) agent** using LangGraph's prebuilt functionality, which combines **reasoning** and **tool usage** in an iterative loop.

## 🔧 **Setup & Configuration**
```python
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)
system_prompt = "You are a helpful assistant that can run Python code."
```
- **LLM**: Databricks Llama 3.3 70B model
- **System prompt**: Defines the agent's behavior

## 🛠️ **Simple Tool Definitions**
```python
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"The weather in {city} is sunny."

def get_time(zone: str) -> str:
    """Get current time for a given timezone."""
    return f"The current time in {zone} is 2:00 PM."
```
- **Simple functions**: Basic weather and time tools
- **Docstrings**: Help the LLM understand what each tool does
- **Mock responses**: Return static responses for demonstration

## 🤖 **ReAct Agent Creation**
```python
agent_graph = create_react_agent(
    model=llm,
    tools=tools,
    prompt=system_prompt
)
```
**Key advantage**: One-line agent creation using LangGraph's prebuilt ReAct implementation

## 🔄 **How ReAct Works**
The ReAct pattern follows this loop:

1. **Thought**: LLM reasons about the user's question
2. **Action**: Decides which tool to use (or if to respond directly)
3. **Observation**: Sees the tool's result
4. **Repeat**: Continues reasoning until it has enough information
5. **Answer**: Provides final response to the user

**Example Flow**:


**Example Flow**:
1. **User**: "What's the weather in Paris and what time is it in Tokyo?"
2. **Thought**: I need to get weather for Paris and time for Tokyo
3. **Action**: get_weather("Paris")
4. **Observation**: "The weather in Paris is sunny"
5. **Thought**: Now I need the time in Tokyo
6. **Action**: get_time("Tokyo") 
7. **Observation**: "The current time in Tokyo is 2:00 PM"
8. **Thought**: I have both pieces of information now
9. **Answer**: "The weather in Paris is sunny and it's 2:00 PM in Tokyo"




## 🎯 **Key Benefits of ReAct Pattern**
- **Iterative reasoning**: Can use multiple tools in sequence
- **Self-correction**: Can reason about tool results and adjust
- **Transparency**: Shows its "thinking" process
- **Flexible**: Can handle complex multi-step problems
- **Automatic**: LangGraph handles the reasoning loop

## 🚀 **Prebuilt Advantage**
Instead of manually building the graph with nodes and edges, `create_react_agent()`:
- **Auto-creates** the reasoning loop
- **Handles** tool calling logic
- **Manages** state transitions
- **Provides** built-in ReAct prompting

This makes creating sophisticated reasoning agents incredibly simple compared to building the graph manually.

### Using langraph create_react_agent

In [0]:
import json
from typing import Any, Generator
from uuid import uuid4

import mlflow
from databricks_langchain import ChatDatabricks, UCFunctionToolkit
from langchain_core.messages import AIMessageChunk
from langgraph.prebuilt import create_react_agent

from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
)

############################################
# Define LLM and tools
############################################
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

# UC_TOOL_NAMES = ["system.ai.python_exec"]
# uc_toolkit = UCFunctionToolkit(function_names=UC_TOOL_NAMES)
# tools = uc_toolkit.tools


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"The weather in {city} is sunny."

def get_time(zone: str) -> str:
    """Get current time for a given timezone."""
    return f"The current time in {zone} is 2:00 PM."


    
  

tools = [get_weather,get_time]

system_prompt = "You are a helpful assistant that can run Python code."

############################################
# Build the ReAct agent automatically
############################################
agent_graph = create_react_agent(
    model=llm,
    tools=tools,
    prompt=system_prompt,  # this injects your system prompt
)





In [0]:
############################################
# Register the agent with MLflow
############################################
from helpers import LangGraphResponsesAgent
mlflow.langchain.autolog()
AGENT = LangGraphResponsesAgent(agent_graph)
mlflow.models.set_model(AGENT)

In [0]:
result = AGENT.predict({"input": [{"role": "user", "content": "What’s the weather in Paris and the time in EST? and search wikipedia for fun facts about paris"}]})
print(result.model_dump(exclude_none=True))

https://python.langchain.com/docs/integrations/tools/

In [0]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


# --- create Wikipedia tool ---
wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [0]:
tools = [get_weather,get_time,wiki]

prompt = """
You are a helpful AI assistant. You can think step-by-step and use the following tools to answer user questions:

1. get_weather(city: str) — returns the current weather in the given city.
2. get_time(zone: str) — returns the current time for the given timezone.
3. WikipediaQueryRun(input: str) — searches Wikipedia and returns relevant information.

Rules:
- If a question needs factual information or background knowledge, prefer WikipediaQueryRun.
- If the user asks about weather, use get_weather.
- If the user asks about time, use get_time.
- Always return a clear final answer to the user after using tools.
- Think carefully about which tool is best for each part of the question. 
- You can use multiple tools in one conversation if the question needs it.
- Do not guess values you can fetch with a tool — always call the correct tool.

Follow the ReAct pattern:
- First, think about what is needed.
- Then, call the right tool with correct arguments.
- Finally, summarize the result back to the user.
"""


############################################
# Build the ReAct agent automatically
############################################
agent_graph = create_react_agent(
    model=llm,
    tools=tools,
    prompt=prompt,  # this injects your system prompt
)


In [0]:
############################################
# Register the agent with MLflow
############################################
from helpers import LangGraphResponsesAgent
mlflow.langchain.autolog()
AGENT = LangGraphResponsesAgent(agent_graph)
mlflow.models.set_model(AGENT)

In [0]:
result = AGENT.predict({"input": [{"role": "user", "content": "What’s the weather in Paris and the time in EST? and search wikipedia for fun facts about paris"}]})
print(result.model_dump(exclude_none=True))